In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd

# Import CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "Chris2025"

db = AnimalShelter(username, password)

df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard - Chris Bevacqua'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
             style={'width':'300px'}),
    html.Hr(),

    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ]),
    
    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    html.Hr(),

    html.Div(style={'display': 'flex'}, children=[
        html.Div(id='graph-id', style={'width': '50%'}),
        html.Div(id='map-id', style={'width': '50%'})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == "water":
        query = {"animal_type": "Dog", "breed": {"$regex": "Labrador|Retriever|Spaniel"}}
    elif filter_type == "mountain":
        query = {"animal_type": "Dog", "breed": {"$regex": "German Shepherd|Husky|Malinois"}}
    elif filter_type == "disaster":
        query = {"animal_type": "Dog"}
    else:
        query = {}

    df = pd.DataFrame.from_records(db.read(query))
    if '_id' in df:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Animal Breeds Distribution')
    return [dcc.Graph(figure=fig)]


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


app.run_server()

Dash app running on https://nirvanaquest-marvinpandora-3000.codio.io/proxy/8050/
